[] uncertainty ana
[] angle ana
[] velo ana




In [1]:
import numpy as np
import znh5md
from ase import units

In [2]:
path  = "data/example_traj.h5"
max_step = 1000
num_mds = 500
cutoff_plane_height = 37.5


frames = znh5md.IO(path)
print(len(frames))

2805


In [3]:
def calc_max_uncertainty(frame, E_uncertainty_limit, F_uncertainty_limit):
    E_max_uncertainty = frame.calc.results['energy_uncertainty']
    F_max_uncertainty = frame.calc.results['forces_uncertainty']
    if E_max_uncertainty > E_uncertainty_limit:
        raise Warning(f'energy uncertainty over {E_uncertainty_limit}')
    if np.any(F_max_uncertainty > F_uncertainty_limit):
        raise Warning(f'energy uncertainty over {F_uncertainty_limit}')
    return E_max_uncertainty, F_max_uncertainty


In [4]:
def magnetude_velo(v):
    mag_v = np.linalg.norm(v, axis=1)
    return mag_v

def angle_in_xy_plane(iv, ov):
    i_angle_rad = np.arctan2(iv[:,1], iv[:,0])
    # i_angle_rad = np.arctan2(iv[1], iv[0])
    i_angle_deg = np.degrees(i_angle_rad)
    
    o_angle_rad = np.arctan2(ov[:,1], ov[:,0])
    # o_angle_rad = np.arctan2(ov[1], ov[0])
    o_angle_deg = np.degrees(o_angle_rad)
    
    for idx, _ in enumerate(o_angle_deg):
        if o_angle_deg[idx] < 0:
            o_angle_deg[idx] = o_angle_deg[idx] + 360
        if i_angle_deg[idx] < 0:
            i_angle_deg[idx] = i_angle_deg[idx] + 360
        
    angle_deg = o_angle_deg - i_angle_deg
    for idx, _ in enumerate(o_angle_deg):
        if angle_deg[idx] < 0:
            angle_deg[idx] = angle_deg[idx] + 360
    return angle_deg, i_angle_deg, o_angle_deg


def polar_angle(v2):
    mag_v2 = np.linalg.norm(v2, axis=1)
    # mag_v2 = np.linalg.norm(v2)

    angle_rad = np.arcsin( v2[:,2] / mag_v2 )
    # angle_rad = np.arcsin( v2[2] / mag_v2 )
    
    angle_deg = np.degrees(angle_rad)

    return angle_deg

In [5]:
####### RICHTIG !!!!!!!!!!!! #############

max_step = 1000
num_mds = 500
cutoff_plane_height = 37.5

E_uncertainty_limit = 15
F_uncertainty_limit = 15

# Counters
total_trajectories = 0
adsorbed_count = 0
reflected_count = 0

# Index tracking
global_frame_idx = 0
trajectory_frame_idx = 0
first_frame_idx = 0
new_trajectory = True

# Velocity data collection
o_velo_vecs = []
i_velo_vecs = []
abs_velos = []

# Main processing loop
for chunk_idx in range(num_mds // 2):
    try:
        chunk_start = chunk_idx * max_step * 2
        chunk_end = (chunk_idx + 1) * max_step * 2
        frames_chunk = frames[chunk_start:chunk_end]
    except:
        frames_chunk = frames[chunk_start:]
        print("Incomplete chunk encountered at end of trajectory.")

    for frame in frames_chunk:
        # Optional: Uncertainty filter (uncomment if needed)
        # _, _ = calc_max_uncertainty(frame, E_uncertainty_limit, F_uncertainty_limit)

        all_below_cutoff = np.all(frame.positions[:, -1] < cutoff_plane_height)
        
        # Only set new_trajectory True if we were in post-reflection state and now all atoms are clearly back under
        if all_below_cutoff and not new_trajectory:
            new_trajectory = True
            first_frame_idx = global_frame_idx
        
        is_reflection = not all_below_cutoff and new_trajectory

        if is_reflection:
            last_frame_idx = global_frame_idx

            # Capture reflected atoms' velocities
            reflected_atoms_idx = np.where(frame.positions[:, -1] > cutoff_plane_height)[0]
            o_velo_vec = frame.get_velocities()[reflected_atoms_idx] * units.s / units.m
            o_velo_vecs.append(o_velo_vec)
            # abs_velos.append(magnetude_velo(o_velo_vec))  # Optional

            # Print reflection summary
            print("\n🪞 Reflected")
            print(f"  Trajectory Length: {trajectory_frame_idx}")
            print(f"  Start Frame: {first_frame_idx} (z = {frames[first_frame_idx].positions[-1, -1]})")
            print(f"  End Frame:   {last_frame_idx} (z = {frames[last_frame_idx].positions[-1, -1]})")

            # Save and reset
            reflected_count += 1
            total_trajectories += 1
            trajectory_frame_idx = 0
            new_trajectory = False
            first_frame_idx = global_frame_idx + 1  # Start next traj on next frame

        elif trajectory_frame_idx == max_step - 1 and new_trajectory:
            last_frame_idx = global_frame_idx

            # Print adsorption summary
            print("\n🧲 Adsorbed")
            print(f"  Trajectory Length: {trajectory_frame_idx + 1}")
            print(f"  Start Frame: {first_frame_idx} (z = {frames[first_frame_idx].positions[-1, -1]})")
            print(f"  End Frame:   {last_frame_idx} (z = {frames[last_frame_idx].positions[-1, -1]})")

            # Save and reset
            adsorbed_count += 1
            total_trajectories += 1
            trajectory_frame_idx = 0
            first_frame_idx = global_frame_idx + 1  # Start next traj on next frame

        elif new_trajectory:
            trajectory_frame_idx += 1

        global_frame_idx += 1

# Final summary
print("\n=== Summary ===")
print(f"Total Frames Processed: {global_frame_idx}")
print(f"Total Trajectories:     {total_trajectories}")
print(f"  Adsorbed:  {adsorbed_count}")
print(f"  Reflected: {reflected_count}")
total_events = adsorbed_count + reflected_count
if total_events > 0:
    print(f"  % Adsorbed:  {100 * adsorbed_count / total_events:.2f}%")
    print(f"  % Reflected: {100 * reflected_count / total_events:.2f}%")

# Sanity check (example frame values)
print("\n=== Sanity Check Frames ===")
print("Frame 2393 z-position:", frames[2393].positions[-1, -1])
print("Frame 2394 z-position:", frames[2394].positions[-1, -1])



🧲 Adsorbed
  Trajectory Length: 1000
  Start Frame: 0 (z = 37.41405149607949)
  End Frame:   999 (z = 26.76557178619477)

🪞 Reflected
  Trajectory Length: 392
  Start Frame: 1000 (z = 37.41405149607949)
  End Frame:   1392 (z = 37.5204904398402)

🧲 Adsorbed
  Trajectory Length: 1000
  Start Frame: 1393 (z = 37.41405149607949)
  End Frame:   2392 (z = 26.626453501617647)

🪞 Reflected
  Trajectory Length: 411
  Start Frame: 2393 (z = 26.540125103541463)
  End Frame:   2804 (z = 37.525322731960294)

=== Summary ===
Total Frames Processed: 2805
Total Trajectories:     4
  Adsorbed:  2
  Reflected: 2
  % Adsorbed:  50.00%
  % Reflected: 50.00%

=== Sanity Check Frames ===
Frame 2393 z-position: 26.540125103541463
Frame 2394 z-position: 37.41405149607949


In [ ]:
n_trajs = 0
idx_of_traj = 0
E_uncertainty_limit = 15
F_uncertainty_limit = 15

n_reflected = 0
n_adsorbed = 0
idx = 0

first_frame_idx = 0
last_frame_idx = None
new_traj_started = True
o_velo_vecs = []
i_velo_vecs = []
abs_velos = []


# Loop through trajectory chunks
for chunk_idx in range(num_mds // 2):
    try:
        frames_ = frames[chunk_idx * max_step * 2 : (chunk_idx + 1) * max_step * 2]
    except:
        frames_ = frames[chunk_idx * max_step * 2 : -1]
        print("x")

    for frame in frames_:
        _, _ = calc_max_uncertainty(frame, E_uncertainty_limit, F_uncertainty_limit)
        
        all_below_cutoff = np.all(frame.positions[:, -1] < cutoff_plane_height)
        new_traj_started = new_traj_started or all_below_cutoff

        if not new_traj_started:
            first_frame_idx = idx
            print('bla')

        additive_reflected = not all_below_cutoff and new_traj_started

        # Check for reflection condition
        if additive_reflected:
            reflected_atoms_idx = np.where(frame.positions[:, -1] > cutoff_plane_height)[0]
            # reflected_atoms_idx = [20, -1]
            
            o_velo_vec = frame.get_velocities()[reflected_atoms_idx] * units.s / units.m
            o_velo_vecs.append(o_velo_vec)
            
            abs_velos.append(magnetude_velo(o_velo_vec))
            
            # print('reflected', idx_of_traj, 
            #       np.linalg.norm(frame.get_velocities()[-1] * units.s / units.m))
        
            last_frame_idx = idx
            print("reflected")
            print(idx_of_traj)
            print(frames[first_frame_idx].get_positions()[-1, -1], first_frame_idx)
            print(frames[last_frame_idx].get_positions()[-1, -1], last_frame_idx)
            first_frame_idx = last_frame_idx+1

            idx_of_traj = -1
            n_trajs += 1
            n_reflected += 1
            new_traj_started = False

        # Check for adsorption condition
        if (idx_of_traj) == (max_step - 1) and not additive_reflected:

            last_frame_idx = idx
            print("adsorbed")
            print(idx_of_traj)
            print(frames[first_frame_idx].get_positions()[-1, -1], first_frame_idx)
            print(frames[last_frame_idx].get_positions()[-1, -1], last_frame_idx)
            first_frame_idx = last_frame_idx+1
   
            idx_of_traj = -1
            n_trajs += 1

            n_adsorbed += 1

        idx += 1
        idx_of_traj += 1
        
print(n_trajs, idx, n_trajs*max_step)
print(len(frames))
frames[0].positions[-1, -1]
print(n_adsorbed, n_reflected, 100/(n_adsorbed+n_reflected)*n_adsorbed, 100/(n_adsorbed+n_reflected)*n_reflected)

print(frames[2393].get_positions()[-1, -1])
print(frames[2394].get_positions()[-1, -1])

adsorbed
999
37.41405149607949 0
26.76557178619477 999
reflected
392
37.41405149607949 1000
37.5204904398402 1392
adsorbed
999
37.41405149607949 1393
26.626453501617647 2392
reflected
411
26.540125103541463 2393
37.525322731960294 2804
4 2805 4000
2805
2 2 50.0 50.0
26.540125103541463
37.41405149607949


In [27]:
999 + 999 + 392 + 411

2801

In [6]:
# idx = [999, 0, 1999, 7]

# db = znh5md.IO('example_traj.h5')
# for i in idx:
#     path = f'reflection_{i}.h5'
#     frames = znh5md.IO(path)[:]
#     db.extend(frames)
    
    
    

In [25]:
j = 0
idx = 0
first = 0
for i in range(4050):
    if (j) == (999):
        last = idx
        print(i)
        print(first, last)
        first = idx+1
        
        j = -1
    
    j += 1
    idx += 1


999
0 999
1999
1000 1999
2999
2000 2999
3999
3000 3999
